# 2. MLE estimation of simple macroeconomic model

In [23]:
import numpy as np
import pandas as pd

In [31]:
mcseries = ms = pd.read_csv("MacroSeries.txt", header = None,squeeze=True, names = ["c","k","w","r"])

c = mcseries["c"]
k = mcseries["k"]
w = mcseries["w"]
r = mcseries["r"]



### (a)

In [32]:
import scipy.stats as sts


In [53]:
# We derive the z in each period that makes
# the characterizing equation hold:
def zvia3(w, k, alpha):
    return np.log(w/((1 - alpha) * (k**alpha)))
    
def eq3z_pdf(z, rho, mu, sigma):
    pdf_vals = np.empty_like(z)
    pdf_vals[0] = sts.norm.pdf(z[0], mu, sigma)
    pdf_vals[1:] = sts.norm.pdf(z[1:], rho * z[:-1] + (1 - rho) * mu, sigma)
    pdf_vals[pdf_vals == 0] = 1e-20
    return pdf_vals
    

    
def log_lik_eq3z(w, k, alpha, rho, mu, sigma):
    
    pdf_vals = eq3z_pdf(zvia3(w, k, alpha), rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit3(params, *args):
    alpha, rho, mu, sigma = params
    w, k = args
    log_lik_val = log_lik_eq3z(w, k, alpha, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

In [54]:
# Guess alpha: 0.3 (standard in literature)

galpha = 0.3

gz = zvia3(w, k, galpha)
gz.describe()

count    100.000000
mean      11.754422
std        0.156162
min       11.480309
25%       11.639685
50%       11.734373
75%       11.843314
max       12.203338
dtype: float64

In [55]:
diffgz = gz.shift(1)-gz
diffgz.describe()

count    99.000000
mean     -0.001842
std       0.097916
min      -0.282904
25%      -0.069374
50%      -0.008672
75%       0.074106
max       0.271477
dtype: float64

In [56]:
# Estimation

import scipy.optimize as opt

# guesses
galpha, grho, gmu, gsigma = 0.3, 0.7605, gz.mean(), diffgz.std()
params_init = np.array([galpha, grho, gmu, gsigma]) 

results = opt.minimize(crit3, params_init,
                       args = (w, k), method ='L-BFGS-B',
                       bounds= ((0, 1), (0, 1),
                               (0, None), (0, None)))
alpha_3, rho_3, mu_3, sigma_3 = results.x
print(results)

      fun: -96.706908069170694
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.26192390e-02,   8.52651283e-06,   8.98126018e-04,
         4.20641300e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 165
      nit: 29
   status: 0
  success: True
        x: array([ 0.45751431,  0.72049086,  9.52274856,  0.09199625])


In [58]:
results.hess_inv.todense()

array([[  1.45716876e-01,  -5.28724869e-01,  -2.14336327e+00,
          3.82582450e-02],
       [ -5.28724869e-01,   3.41017995e+00,   8.49493542e+00,
         -2.88586802e-01],
       [ -2.14336327e+00,   8.49493542e+00,   3.19011562e+01,
         -6.36095903e-01],
       [  3.82582450e-02,  -2.88586802e-01,  -6.36095903e-01,
          2.52159143e-02]])

### (b)

In [60]:
# We derive the z in each period that makes
# the characterizing equation hold:
def zvia4(r, k, alpha):
    return np.log(r/((alpha) * (k**(alpha - 1))))
    
def eq4z_pdf(z, rho, mu, sigma):
    pdf_vals = np.empty_like(z)
    pdf_vals[0] = sts.norm.pdf(z[0], mu, sigma)
    pdf_vals[1:] = sts.norm.pdf(z[1:], rho * z[:-1] + (1 - rho) * mu, sigma)
    pdf_vals[pdf_vals == 0] = 1e-20
    return pdf_vals
    

    
def log_lik_eq4z(r, k, alpha, rho, mu, sigma):
    
    pdf_vals = eq4z_pdf(zvia4(r, k, alpha), rho, mu, sigma)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit4(params, *args):
    alpha, rho, mu, sigma = params
    r, k = args
    log_lik_val = log_lik_eq3z(r, k, alpha, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

In [61]:
galpha = 0.3

gz4 = zvia4(r, k, galpha)
gz4.describe()

count    100.000000
mean      12.278946
std        0.156162
min       12.004833
25%       12.164209
50%       12.258897
75%       12.367839
max       12.727862
dtype: float64

In [62]:
diffgz4 = gz4.shift(1)-gz4
diffgz4.describe()

count    99.000000
mean     -0.001842
std       0.097916
min      -0.282904
25%      -0.069374
50%      -0.008672
75%       0.074106
max       0.271477
dtype: float64

In [72]:
# Estimation

import scipy.optimize as opt

# guesses
galpha, grho, gmu, gsigma = 0.3, 0.7605, gz.mean(), diffgz.std()
params_init = np.array([galpha, grho, gmu, gsigma]) 

results = opt.minimize(crit4, params_init,
                       args = (w, k), method ='L-BFGS-B',
                       bounds= ((0, 1), (0, 1),
                               (0, None), (0, None)))
alpha_4, rho_4, mu_4, sigma_4 = results.x
print(results)

      fun: -96.706908069170694
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  1.26192390e-02,   8.52651283e-06,   8.98126018e-04,
         4.20641300e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 165
      nit: 29
   status: 0
  success: True
        x: array([ 0.45751431,  0.72049086,  9.52274856,  0.09199625])


In [73]:
results.hess_inv.todense()

array([[  1.45716876e-01,  -5.28724869e-01,  -2.14336327e+00,
          3.82582450e-02],
       [ -5.28724869e-01,   3.41017995e+00,   8.49493542e+00,
         -2.88586802e-01],
       [ -2.14336327e+00,   8.49493542e+00,   3.19011562e+01,
         -6.36095903e-01],
       [  3.82582450e-02,  -2.88586802e-01,  -6.36095903e-01,
          2.52159143e-02]])

Note here that we used the same initial conditions as before. Using more appropriate ones,...

In [74]:

import scipy.optimize as opt

# guesses
galpha, grho, gmu, gsigma = 0.3, 0.7605, 12, diffgz4.std()
params_init = np.array([galpha, grho, gmu, gsigma]) 

results = opt.minimize(crit4, params_init,
                       args = (w, k), method ='L-BFGS-B',
                       bounds= ((0, 1), (0, 1),
                               (0, None), (0, None)))
alpha_4, rho_4, mu_4, sigma_4 = results.x
print(results)


      fun: -96.706908082520158
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.00840146, -0.00086118, -0.0005528 , -0.00472085])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 220
      nit: 38
   status: 0
  success: True
        x: array([ 0.45753617,  0.72047056,  9.52244144,  0.09199603])


In [75]:
results.hess_inv.todense()

array([[  4.37253648e+01,  -1.82742147e+01,  -6.11791607e+02,
          1.99729996e+00],
       [ -1.82742147e+01,   7.88278477e+00,   2.55743265e+02,
         -8.39522932e-01],
       [ -6.11791607e+02,   2.55743265e+02,   8.56001134e+03,
         -2.79465737e+01],
       [  1.99729996e+00,  -8.39522932e-01,  -2.79465737e+01,
          9.14299925e-02]])

... gives us an almost identical estimate for the parameters. We would need to do more testing to assess robustness of this result, but the similarity despite different starting conditions gives us confidence.

### (c)

As hinted, we first use Eq4 to get a cutoff value for z:

In [77]:
z_cutoff = zvia4(1,7500000, alpha_3)
z_cutoff

9.3697199757214573

In [79]:
prob = 1 - sts.norm.cdf(z_cutoff, rho_3*10 + (1 - rho_3) * mu_3, sigma_3)
prob

0.99999996688916082

We can see that the probability that the interest rate will be greater than one is approximately 1.

In [83]:
z_cutoff = zvia4(1,7500000, alpha_4)
z_cutoff

9.369326171490421

In [84]:

prob = 1 - sts.norm.cdf(z_cutoff, rho_4*10 + (1 - rho_4) * mu_4, sigma_4)
prob

0.99999996748478981

The result also seems fairly robust across the estimates. For more confidence, further testing is necessary.